In [1]:
import praw
from praw.models import MoreComments
import pandas as pd

In [2]:
reddit = praw.Reddit(
    
        )

In [11]:
'''
Paramter: user - a redditor object

Return: True if user is not an automod or a bot
'''
def is_bot(user):
    return user.name.lower() == 'automoderator' or user.name.lower() == "bot"

In [12]:
def is_valid_post(post):
    """
    Returns True if the post has 4 or more words, is not an image, is not only a link, and is not made by a bot or Automoderator.
    """
    if post.is_self:  # Check if the post is a text post
        # Split the post text into words and check if the length of the resulting list is >= 4
        if len(post.selftext.split()) >= 4:
            return True
    elif post.url.endswith('.jpg') or post.url.endswith('.png'):  # Check if the post is an image
        return False
    elif post.url == post.permalink:  # Check if the post is only a link to itself
        return False
    else:
        user = post.author
        if user and (user.name.lower() == 'automoderator' or 'bot' in user.name.lower()):
            return False
        else:
            return True


In [5]:
'''
Parameters: ticker - ticker to search for
            subreddit - subreddit to search through
            user_dict - dictionary of user with posts mentioning ticker
            time - how far back to search (day, week, month, year)
'''
def search_for_ticker_posts(ticker, subreddit, user_dict, time_frame):
    
    search_results = subreddit.search(f'title:{ticker}', time_filter=time_frame)
    
    #Loops through each post that matches the search result
    for post in search_results:
        
        if is_valid_post(post):
            author = post.author.name
            post_id = post.id
                
            #Checks if author is already in dictionary
            if (author not in user_dict):
                user_dict[author] = []
                    
            if post_id not in user_dict[author]:
                user_dict[author].append(post_id)


In [6]:
'''
Parameters: tickers - list of tickers to search for
            subreddit - list of subreddits to search through
            time_frame - how far back to search (day, week, month, year)
            
Returns: user_dict - a dictionary of users who mentioned tickers in list of subs, their post_ids as values
'''
def search_from_subs(tickers, subreddits, time_frame):
    
    user_dict = {}
    
    for sub in subreddits:
        for ticker in tickers:
            
            search_for_ticker_posts(ticker, sub, user_dict, time_frame)
    
    return user_dict

In [7]:
wsb = reddit.subreddit("wallstreetbets")
invest = reddit.subreddit("investing")
stocks = reddit.subreddit("stocks")
daytrade = reddit.subreddit("daytrading")

subreddits =  [wsb, invest, stocks, daytrade]

In [8]:
tsla_tickers = ['TSLA', 'Tesla', 'tesla', 'TESLA']
appl_tickers = ['AAPL', 'APPLE', 'Apple','apple']
amd_tickers = ['AMD', 'amd']
msft_tickers = ['MSFT', 'MICROSOFT', 'Microsoft']
amzn_tickers = ['AMZN', 'Amazon', 'amazon', 'AMAZON']
nvda_tickers = ['NVDA','nvda', 'NVDIA', 'Nvdia']
nflx_tickers = ['NFLX', 'nflx', 'NETFLIX', 'Netflix']
meta_tickers = ['META', 'Meta', 'meta']

In [9]:
tsla_dict= search_from_subs(tsla_tickers, subreddits, "year")

In [10]:
appl_dict= search_from_subs(appl_tickers, subreddits, "year")

In [11]:
amd_dict= search_from_subs(amd_tickers, subreddits, "year")

In [12]:
msft_dict= search_from_subs(msft_tickers, subreddits, "year")

In [13]:
amzn_dict= search_from_subs(amzn_tickers, subreddits, "year")

In [14]:
nvda_dict= search_from_subs(nvda_tickers, subreddits, "year")

In [15]:
nflx_dict= search_from_subs(nflx_tickers, subreddits, "year")

In [16]:
meta_dict= search_from_subs(meta_tickers, subreddits, "year")

In [17]:
dict_list = [appl_dict, tsla_dict, amd_dict, amzn_dict, msft_dict, nvda_dict, nflx_dict, meta_dict]

In [18]:
tickers = ['AAPL', 'TSLA', 'AMD', 'AMZN', 'MSFT', 'NVDA', 'NFLX', 'META']

In [22]:
def create_user_post_df(dict_list):
    # Create a set of all unique usernames
    all_users = set()
    for dictionary in dict_list:
        all_users.update(dictionary.keys())

    # Create a dictionary with each username as the key and an empty dictionary as the value
    user_post_dict = {user: {} for user in all_users}

    # Iterate through each dictionary in the list
    for i, dictionary in enumerate(dict_list):
        # Iterate through each user in the dictionary
        for user in dictionary:
            # Iterate through each post_id for the user
            for post_id in dictionary[user]:
                # Add the dictionary name as a column and the post_id as a value in the corresponding row
                user_post_dict[user].setdefault(i, []).append(post_id)

    # Convert the user_post_dict to a DataFrame
    df = pd.DataFrame.from_dict(user_post_dict, orient='index')
    # Rename the columns with the corresponding dictionary names
    df.columns = [f"dict_{i}" for i in range(len(dict_list))]

    return df

In [23]:
df = create_user_post_df(dict_list)

In [24]:
df

,dict_0,dict_1,dict_2,dict_3,dict_4,dict_5,dict_6,dict_7
hottewhells,[12iqzh0],NaN,NaN,NaN,NaN,NaN,NaN,NaN
bbportali,[vyykio],NaN,NaN,NaN,[vyykio],NaN,NaN,NaN
MacroDragon1,[zk65n5],NaN,NaN,NaN,NaN,NaN,NaN,NaN
adamzito1,[w8xdp8],NaN,NaN,NaN,NaN,NaN,NaN,NaN
lawthrowaway101,[13dywvy],NaN,"[121q0hp, 12u30we, 12go0l1, 13rw52p, 1390tva, ...",[131cpjh],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
Neofire9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[wec2q3]
RUFUSINS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[wbqyqu]
Pahlawan_Ketupat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[119kpwa]
jamtradez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[xxge0l]


In [40]:
df.to_csv('data.csv', index=True)

In [3]:
df_two = pd.read_csv('data.csv',index_col=0)
df_two

,dict_0,dict_1,dict_2,dict_3,dict_4,dict_5,dict_6,dict_7
hottewhells,['12iqzh0'],NaN,NaN,NaN,NaN,NaN,NaN,NaN
bbportali,['vyykio'],NaN,NaN,NaN,['vyykio'],NaN,NaN,NaN
MacroDragon1,['zk65n5'],NaN,NaN,NaN,NaN,NaN,NaN,NaN
adamzito1,['w8xdp8'],NaN,NaN,NaN,NaN,NaN,NaN,NaN
lawthrowaway101,['13dywvy'],NaN,"['121q0hp', '12u30we', '12go0l1', '13rw52p', '...",['131cpjh'],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
Neofire9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['wec2q3']
RUFUSINS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['wbqyqu']
Pahlawan_Ketupat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['119kpwa']
jamtradez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['xxge0l']


In [41]:
notna_values = df_two['dict_6'].notna()
result = df_two.loc[notna_values, 'dict_6']
print(result)

TradeTheZones                               ['13sowgy']
Guysmarket              ['13ne5be', 'yx6rl6', 'yx0gth']
PsychologicalTest781                         ['ywwwbv']
DaddyDersch                      ['13r0uu2', '13r0wga']
NoobInvester018                             ['13mfz75']
                                     ...               
cbusoh66                                    ['13rv7ov']
1ncehost                                    ['13lb98n']
Objective-Service324             ['13qxrr4', '13rj5sh']
iBeFlying676                                ['13qzr16']
echisholm                                   ['11g2pc2']
Name: dict_6, Length: 94, dtype: object


In [42]:
reddit.submission(id="13rj5sh").title

'30k on NVDA'

dict_0 = MSFT, dict_1= AAPL,  dict_2 = TSLA, dict_3 = AMZN, dict_4 = NFLX, dict_5 = META, dict_6 = NVDA dict_7 = AMD

In [43]:
new_columns = ['$MSFT', '$AAPL', '$TSLA', '$AMZN', '$NFLX', '$META', '$NVDA', '$AMD']

In [46]:
df_two = df_two.rename(columns=dict(zip(df_two.columns, new_columns)))

df_two

,$MSFT,$AAPL,$TSLA,$AMZN,$NFLX,$META,$NVDA,$AMD
hottewhells,['12iqzh0'],NaN,NaN,NaN,NaN,NaN,NaN,NaN
bbportali,['vyykio'],NaN,NaN,NaN,['vyykio'],NaN,NaN,NaN
MacroDragon1,['zk65n5'],NaN,NaN,NaN,NaN,NaN,NaN,NaN
adamzito1,['w8xdp8'],NaN,NaN,NaN,NaN,NaN,NaN,NaN
lawthrowaway101,['13dywvy'],NaN,"['121q0hp', '12u30we', '12go0l1', '13rw52p', '...",['131cpjh'],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
Neofire9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['wec2q3']
RUFUSINS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['wbqyqu']
Pahlawan_Ketupat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['119kpwa']
jamtradez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['xxge0l']


In [48]:
df_two.to_csv('labeled-data.csv', index=True)

In [3]:
data = pd.read_csv('labeled-data.csv',index_col=0)
data

,$MSFT,$AAPL,$TSLA,$AMZN,$NFLX,$META,$NVDA,$AMD
hottewhells,['12iqzh0'],NaN,NaN,NaN,NaN,NaN,NaN,NaN
bbportali,['vyykio'],NaN,NaN,NaN,['vyykio'],NaN,NaN,NaN
MacroDragon1,['zk65n5'],NaN,NaN,NaN,NaN,NaN,NaN,NaN
adamzito1,['w8xdp8'],NaN,NaN,NaN,NaN,NaN,NaN,NaN
lawthrowaway101,['13dywvy'],NaN,"['121q0hp', '12u30we', '12go0l1', '13rw52p', '...",['131cpjh'],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
Neofire9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['wec2q3']
RUFUSINS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['wbqyqu']
Pahlawan_Ketupat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['119kpwa']
jamtradez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['xxge0l']


In [9]:
filtered_df = data[data.iloc[:, 1:].notnull().sum(axis=1) >= 2]
filtered_df

,$MSFT,$AAPL,$TSLA,$AMZN,$NFLX,$META,$NVDA,$AMD
lawthrowaway101,['13dywvy'],NaN,"['121q0hp', '12u30we', '12go0l1', '13rw52p', '...",['131cpjh'],NaN,NaN,NaN,NaN
Astroman023,['10jwkgq'],NaN,NaN,NaN,['10fvp9u'],['10nftbp'],NaN,NaN
JMichael12T,['x9fjzc'],NaN,NaN,['x9pizx'],NaN,NaN,NaN,['xbojy7']
Guysmarket,"['w69hk8', 'vaoptj', 'w91vth', 'vaevci']","['137z7xg', 'x6oms2', 'vfgi4j', 'z68fnt', 'vfy...","['12s8cfb', 'vaevci']","['vaoptj', 'vaevci']",['w3cwti'],NaN,"['13ne5be', 'yx6rl6', 'yx0gth']",['wexhxz']
predictany007,"['13igt6u', 'w9cih5', '12zedri', '13j5014', '1...","['y83owq', '13rl2eq', '136w8h8', '138ikrk', 'z...","['13i691q', '12rq3pf', '12svh8b', '120qykr', '...","['13qudsg', '13rl2eq', 'z122gg', 'zya1j1', 'yv...","['12rqd26', '13pwxdb', '13l69lg', '10gdagf', '...","['13ppjis', 'yldmuu', 'v44p5o']",NaN,"['xy1yx2', 'weh5zm']"
...,...,...,...,...,...,...,...,...
putsandcalls,NaN,NaN,NaN,NaN,NaN,NaN,['13razs0'],['13razs0']
RememberYo,NaN,NaN,NaN,NaN,NaN,NaN,['119g7lv'],['x1w0ig']
Virus4762,NaN,NaN,NaN,NaN,NaN,NaN,['wfp8rt'],['wfp8rt']
zerof3565,NaN,NaN,NaN,NaN,NaN,NaN,['13a5v9i'],['13a5v9i']


In [10]:
filtered_df.to_csv('labeled-data.csv', index=True)